In [1]:
import pandas as pd
df = pd.read_csv("top5-players.csv")
print(df.shape)
print(df.isna().sum())

(2852, 37)
Rk             0
Player         0
Nation         3
Pos            0
Squad          0
Comp           0
Age            4
Born           4
MP             0
Starts         0
Min            0
90s            0
Gls            0
Ast            0
G+A            0
G-PK           0
PK             0
PKatt          0
CrdY           0
CrdR           0
xG             1
npxG           1
xAG            1
npxG+xAG       1
PrgC           1
PrgP           1
PrgR           1
Gls_90         0
Ast_90         0
G+A_90         0
G-PK_90        0
G+A-PK_90      0
xG_90          1
xAG_90         1
xG+xAG_90      1
npxG_90        1
npxG+xAG_90    1
dtype: int64


In [ ]:
import pandas as pd
df = pd.read_csv("top5-players.csv")
df = df.drop_duplicates()
df = df.dropna()
df.to_csv("top5playersclean.csv", index=False)



(2846, 37)
